# Shamir Finite Field

This script is a variant of *Shamir.ipynb*
It use the Shamir's sharing-secret schemes to encode a secret and decode a secret on modular field

## How does it work ?

A modular field $A$ modulo $n$ is a field that looks like :
$$
\{0,1,2,3,4,5\dots n-1\}
$$
When $n-1$ is exceeded it restart at $0$

For example : $A$ is a field$\mod \ 5$
So it understand any number in : 
$$\{0,1,2,3,4\}$$
If we want the sum from $0$ to $6\bmod 5$ :
$$0+1+2+3+4+5+6 \ (\bmod \ 5)$$

$$=0+1+2+3+4+0+1$$
So : $6 \ (\bmod \ 5)=1$ , $7 \ (\bmod \ 5)=2 \dots$

So like in *Shamir.ipynb* the script create a function $f$ and find points modulo $n$ for encoding and find $f$ and the secret modulo $n$ for decoding.


### coder_finite_field()

This function encode the secret

First it needs 4 arguments :

`` coder( n, k, s, mod )``

$n$ is the number of share that we want to create.
$k$ is the number of shares that is needed to find the secret.
$s$ is the secret.
$\bmod$ is the modulo of the finite field on which points are created.

First the algorithm creates a function $f$ with $s$ as the first number than add random coefficient. ``k-1`` correspond to the degree of the polynomial.
```
f = [secret]
        pts = []
        ptX = 0
        for i in range(k-1):
            f.append(random.randint(1,mod))
        
```

Then it add random $x$ position and calculate $f(x)\bmod 5$.

```
used_ptX = []
for i in range(n):
	while ptX in used_ptX:
	    ptX = random.randint(1,mod-1)
	ptY = 0
	degre = 0
	for i in f:
	    ptY = (ptY + i*(ptX**degre))%mod
	    degre += 1
	used_ptX.append(ptX)
  pts.append("("+str(ptX)+";"+str(ptY)+")")
```
Last it add the coordinates to the *data.txt* file and print the different point

```
fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "w")
for i in pts:
    print(i)
    fichier.write(i+"\n")

In [8]:
import random
def coder_finite_field(n,k,secret,mod): #avec k le nombre de points nécessaires pour décoder et n le nombre de points partagés
    if k <= 0 or mod == 0 or mod < n or mod < secret:
        print("Erreur, cette valeur n'est pas valable")
    else :
        f = [secret]
        pts = []
        ptX = 0
        used_ptX = [0]
        for i in range(k-1):
            f.append(random.randint(1,mod))
        
        for i in range(n):
            while ptX in used_ptX:
                ptX = random.randint(1,mod-1)
            ptY = 0
            degre = 0
            for i in f:
                ptY = (ptY + i*(ptX**degre))%mod
                degre += 1
            
            used_ptX.append(ptX)
            pts.append("("+str(ptX)+";"+str(ptY)+")")
        print("Voici les différents points à distribuer à chaques personnes :")
        fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "w")
        for i in pts:
            print(i)
            fichier.write(i+"\n")
        

### decoder()

This function decode the secret.

First it needs 2 arguments :

`` decoder( k, mod )``

$k$ is the number of shares that is needed to find the secret.
$\bmod$ is the modulo of the finite field on which points are created.

It first take only the points needed in *data.txt*
```
fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
a = fichier.read()
fichier.close()
pts = []
coordonnes = []
for i in a :
    coordonnes.append(i)
v = 0
signe = 1
for i in coordonnes:
    if i in ["1","2","3","4","5","6","7","8","9","0"]:
        v = v*10 + int(i)
    elif i == "-":
        signe = -1
    elif i == ";" or i == ")":
        v = v * signe
        pts.append(v%mod)
        signe = 1
        v = 0
```

Then it solve the system with the Matrix and print the secret.
The code is explicated in *Gauss_Matrix_Finite_Field.ipynb*.
```
if k <= 0:
    return "Erreur, une des valeurs n'est pas valable"
else:
    if len(pts)-2*k < 0:
        return "Les points fournis ne sont pas suffisants"
    for i in range(len(pts)-2*k):
        del pts[0]
    Matrice_Gauss = []
    for c in range(k):
        for l in range(k):
            Matrice_Gauss.append((pts[2*c]**l)%mod)
        Matrice_Gauss.append((pts[2*c+1])%mod)


    impossibilite = 0
    for colonneactive in range(k):
        
        pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
        inverse_pivot = 0
        while (inverse_pivot * pivot)%mod != 1:
            inverse_pivot += 1
            if inverse_pivot == mod**2:
                
                return "Impossible"

        if pivot == 0:
            impossibilite = 1
            break
        for l in range(k+1):
            
            Matrice_Gauss[colonneactive*(k+1)+l] = (Matrice_Gauss[colonneactive*(k+1)+l]*inverse_pivot)%mod


        for c in range(k):
            coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
            for l in range(k+1):
                if c != colonneactive:
                    Matrice_Gauss[c*(k+1)+l]=(Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne)%mod
    
    if impossibilite == 1:
        return "Les points fournis ne sont pas suffisants"
    else:
        print( "Avec les points :\n",a,"\n Le secret est : ",Matrice_Gauss[k], sep = "")
        return Matrice_Gauss[k]
```


In [ ]:
def decoder_finite_field(k,mod):
    fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
    a = fichier.read()
    fichier.close()
    pts = []
    coordonnes = []
    for i in a :
        coordonnes.append(i)
    v = 0
    signe = 1
    for i in coordonnes:
        if i in ["1","2","3","4","5","6","7","8","9","0"]:
            v = v*10 + int(i)
        elif i == "-":
            signe = -1
        elif i == ";" or i == ")":
            v = v * signe
            pts.append(v%mod)
            signe = 1
            v = 0
    


    if k <= 0:
        return "Erreur, une des valeurs n'est pas valable"
    else:
        if len(pts)-2*k < 0:
            return "Les points fournis ne sont pas suffisants"
        for i in range(len(pts)-2*k):
            del pts[0]
        Matrice_Gauss = []
        for c in range(k):
            for l in range(k):
                Matrice_Gauss.append((pts[2*c]**l)%mod)
            Matrice_Gauss.append((pts[2*c+1])%mod)


        impossibilite = 0
        for colonneactive in range(k):
            
            pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
            inverse_pivot = 0
            while (inverse_pivot * pivot)%mod != 1:
                inverse_pivot += 1
                if inverse_pivot == mod**2:
                    
                    return "Impossible"

            if pivot == 0:
                impossibilite = 1
                break
            for l in range(k+1):
                
                Matrice_Gauss[colonneactive*(k+1)+l] = (Matrice_Gauss[colonneactive*(k+1)+l]*inverse_pivot)%mod

            for c in range(k):
                coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
                for l in range(k+1):
                    if c != colonneactive:
                        Matrice_Gauss[c*(k+1)+l]=(Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne)%mod
        
        if impossibilite == 1:
            return "Les points fournis ne sont pas suffisants"
        else:
            print( "Avec les points :\n",a,"\n Le secret est : ",Matrice_Gauss[k], sep = "")
            return Matrice_Gauss[k]
            

            
        


In [ ]:
coder_finite_field(4,3,3,5)

In [ ]:
decoder_finite_field(3,5)